# **Sistema di riconoscimento visivo tramite FuseMedML**



## **Fase di settaggio dell'ambiente**

### Montaggio delle cartelle di Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Clonazione e installazione della libreria FuseMedML

In [ ]:
!git clone https://github.com/IBM/fuse-med-ml.git     #clonazione della repository delle funzioni di fuse
%cd fuse-med-ml                                       
!pip install -e .                                     #aggiornamento delle dipendenze pip per l'elaborazione delle funzioni

Cloning into 'fuse-med-ml'...
remote: Enumerating objects: 3915, done.
remote: Counting objects: 100% (1154/1154), done.
remote: Compressing objects: 100% (603/603), done.
remote: Total 3915 (delta 607), reused 1011 (delta 533), pack-reused 2761
Receiving objects: 100% (3915/3915), 74.59 MiB | 36.18 MiB/s, done.
Resolving deltas: 100% (2231/2231), done.
/content/fuse-med-ml
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/fuse-med-ml
     |████████████████████████████████| 38.1 MB 1.3 MB/s 
     |████████████████████████████████| 11.2 MB 50.4 MB/s 
     |████████████████████████████████| 48.4 MB 19 kB/s 
     |████████████████████████████████| 2.0 MB 44.4 MB/s 
     |████████████████████████████████| 9.7 MB 28.0 MB/s 
     |████████████████████████████████| 69 kB 9.2 MB/s 
     |████████████████████████████████| 212 kB 59.3 MB/s 
     |████████████████████████████████| 930 kB 57.3 MB/s 
     |██████████████████

### Import di Librerie Python e Fuse

In [ ]:
import os
from typing import OrderedDict

import torch
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torch.utils.data.dataloader import DataLoader
from torchvision import transforms, datasets
from sklearn.model_selection import train_test_split

from fuse.eval.evaluator import EvaluatorDefault
from fuse.data.dataset.dataset_wrapper import FuseDatasetWrapper
from fuse.data.sampler.sampler_balanced_batch import FuseSamplerBalancedBatch
from fuse.losses.loss_default import FuseLossDefault
from fuse.managers.callbacks.callback_tensorboard import FuseTensorboardCallback
from fuse.managers.manager_default import FuseManagerDefault
from fuse.eval.metrics.classification.metrics_classification_common import MetricAccuracy, MetricAUCROC, MetricROCCurve, MetricAUCPR, MetricConfusionMatrix, MetricBSS
from fuse.eval.metrics.classification.metrics_thresholding_common import MetricApplyThresholds
from fuse.models.model_wrapper import FuseModelWrapper
from fuse_examples.tutorials.hello_world.hello_world_utils import LeNet, perform_softmax
from fuse.data.augmentor.augmentor_toolbox import aug_image_default_pipeline

### Definizione dei path di output


In [ ]:
ROOT = 'CCS' # Cartella che conterrà tutti i file necessari al funzionamento della rete
PATHS = {'model_dir': os.path.join(ROOT, 'T1VOL/model_dir'),
         'force_reset_model_dir': True,  # Se impostato a True il path contenente il modello verrà ripristinato automaticamente - 
                                         # altrimenti è necessario ogni volta confermare l'operazione di ripristino tramite comando.
         'cache_dir': os.path.join(ROOT, 'T1VOL/cache_dir'),
         'inference_dir': os.path.join(ROOT, 'T1VOL/infer_dir'),
         'eval_dir': os.path.join(ROOT, 'T1VOL/eval_dir')}

paths = PATHS

## **Fase di settaggio dei parametri di addestramento**

### Parametri generici di addestramento

All'interno della libreria Fuse, è necessario settare alcune tipoligie obbligatorie di parametri, tra cui si possono distinguere tre differenti classi:
* Parametri di tipo **Model** - che tipo di modello si utilizza.
* Parametri di tipo **Data** - definisce i parametri per il preprocessing.
* Parametri di tipo **Manager** - definisce i parametri per il training.

In [ ]:
TRAIN_COMMON_PARAMS = {}

### Model ###
TRAIN_COMMON_PARAMS['model'] = 'vgg11'                    #modello scelto: VGG11

### Data ###
TRAIN_COMMON_PARAMS['data.batch_size'] = 70               #dimensione di ogni batch
TRAIN_COMMON_PARAMS['data.train_num_workers'] = 8         #numero di worker della rete durante il training
TRAIN_COMMON_PARAMS['data.validation_num_workers'] = 8    #numero di worker della rete durante la validazione

### Manager ###
TRAIN_COMMON_PARAMS['manager.train_params'] = {
    'device': 'cuda',                 # device, si prende la scheda video
    'num_epochs': 40,                 # numero di epoche durante la fase di training
    'virtual_batch_size': 1,          # numero di batch in un batch virtuale: in questo caso la mappatura è 1:1
    'start_saving_epochs': 5,         # prima epoca da cui comincio a salvare i pesi
    'gap_between_saving_epochs': 2,   # numero di epoche tra ogni checkpoint di pesi
                                      # ogni 5 epoche salvo i pesi della rete, partendo dall'epoca n.10
}
TRAIN_COMMON_PARAMS['manager.best_epoch_source'] = {
    'source': 'metrics.accuracy',     # si sceglie la metrica di valutazione dal dizionario 'epoch_result': in questo caso Accuracy
    'optimization': 'max',            # si sceglie l'obiettivo per tale metrica, in questo caso si vuole massimizzare l'accuracy
    'on_equal_values': 'better',      # si sceglie che cosa fare in corrispondenza di valori di accuracy uguali nella best epoch, 
                                      # in questo caso si prende la 'better', ma potevo scegliere anche 'worst'
}
TRAIN_COMMON_PARAMS['manager.learning_rate'] = 0.0001               #si definisce il learning rate
TRAIN_COMMON_PARAMS['manager.weight_decay'] = 0.001                 #si definisce il decay dei pesi della rete
TRAIN_COMMON_PARAMS['manager.resume_checkpoint_filename'] = None    # Messo a None prova a ripristinare il checkpoint

TRAIN_COMMON_PARAMS['manager.train_params']['device'] = 'cuda'   # si sceglie il device su cui eseguire la rete

train_params = TRAIN_COMMON_PARAMS

#### Dimensione virtuale dei batch

Per i modelli le cui prestazioni sono limitate dalla memoria della GPU, e quindi dalla dimensione dei batch - molti modelli NLP, in particolare, hanno questo problema - questa semplice tecnica offre un modo semplice per ottenere una dimensione "virtuale" dei batch più grande di quella che si adatta alla memoria.
Per esempio, se è possibile inserire solo 16 campioni per batch nella memoria della GPU, è possibile inoltrare due batch, poi passare all'indietro una volta, per una dimensione effettiva di 32 batch. Oppure passare avanti quattro volte, passare indietro una volta, per una dimensione del batch di 64. E così via.
Questo è possibile impostarlo tramite fuse variando il parametro 'virtual_batch_size', che indica il numero di batch effettivi da includere all'interno di un batch virtuale

#### Decadimento dei pesi

Il parametro 'weight_decay' serve a stabilire un coefficiente di penalità per il learning rate. Questo parametro viene aggiunto alla loss calcolata al passo precedente, moltiplicando tale fattore per la norma quadra dei pesi precedente. In sostanza, viene usata la formula:
$$loss(i)=loss(i-1) + WD*||weights||^2$$

## **Fase di processing dei dati**
Si vanno a convertire in dataloaders tutti i dati presenti, sfruttanto la funzione di pytorch (`torch.utils.data.DataLoader`) sia per la parte di validation che per la parte di training usando i seguenti componenti Fuse:
1. Wrapper - **FuseDatasetWrapper**:
    Raccoglie il dataset convertito in DataLoader in un dizionario tale che sia mappato con le etichette date in input.
2. Sampler - **FuseSamplerBalancedBatch**:
    Implementa semplicemente il sampler di Pytorch 'torch.utils.data.sampler'. Tale sampler crea dei batch bilanciati tra le classi, comprendendo un uguale numero di samples per ogni classe all'interno del batch.

In [ ]:
transform = transforms.Compose([                        #si va a definire una trasformazione in tensori
    transforms.Resize((224,224)),                       #si ridefinisce la dimensione dell'immagine
    transforms.ToTensor(),                              #si attua la trasformazione in tensore
    transforms.Normalize((0.1307,), (0.3081,))          #si applica una normalizzazione secondo dei pesi prefissati
])

In [ ]:
!pip install split-folders                                                                  #libreria per l'installazione della funzioni di split
import splitfolders

data_dir = '/content/drive/MyDrive/DATI/T1VOL'                                             #directory contenente i dati da partizionare
splitfolders.ratio(data_dir, output="content/DATASET", seed=1337, ratio=(.6, 0.2, 0.2))     #creazione dello split con la definizione delle percentuali

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Copying files: 2264 files [00:29, 77.35 files/s] 


In [ ]:
# si definisce il wrapping come descritto prima per il dataset di training, mappando il dataset con un dizionario che contiene 'immagine' e 'etichetta'
data_dir = 'content/DATASET'

#================================================================================================
#                                          TRAINING DATASET
#================================================================================================

torch_train_dataset = {x: datasets.ImageFolder(os.path.join(data_dir, x), transform) for x in ['train', 'val']}
train_dataset = FuseDatasetWrapper(name='train', dataset=torch_train_dataset['train'], mapping=('image', 'label'))

# si procede a creare il dataset wrappato
train_dataset.create()

# si definisce il sampler per la creazione dei batch bilanciati di Fuse
sampler = FuseSamplerBalancedBatch(dataset=train_dataset,                       #si fornisce in input il dataset da cui creare i batch
                                balanced_class_name='data.label',               #si definisce l'etichetta secondo la quale si effettua il bilanciamento
                                num_balanced_classes=2,                         #si imposta il numero di classi da bilanciare
                                batch_size=train_params['data.batch_size'],     #dimensione del batch, che avendo messo none al parametro di dopo voglio che sia diviso per il num_balanced_classes
                                balanced_class_weights=None)                    #mettendo None dico che voglio un numero di classi uguale per ogni batch, altrimenti è un intero 
                                                                                #che definisce quanti campioni di ogni classe vanno in un batch

# Creo il dataloader con la funzione apposita di pytorch
train_dataloader = DataLoader(dataset=train_dataset, batch_sampler=sampler, num_workers=train_params['data.train_num_workers'])


#================================================================================================
#                                         VALIDATION DATASET
#================================================================================================


# faccio il wrapping con la funzione di fuse
validation_dataset = FuseDatasetWrapper(name='validation', dataset=torch_train_dataset['val'], mapping=('image', 'label'))
validation_dataset.create()

# e creo il dataloader con pytorch
validation_dataloader = DataLoader(dataset=validation_dataset, batch_size=train_params['data.batch_size'],
                                num_workers=train_params['data.validation_num_workers'])

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


## **Fase di definizione del modello**
Si construisce ora la rete VGG11 usando PyTorch e poi se ne fa il wrapping usando le funzioni di Fuse.
Il modello di output sarà aggregato in un dizionario chiamato `batch_dict['model.*']`.

In [ ]:
from torchvision.models import vgg11

torch_model = vgg11(pretrained = True)                                                      #prendo il modello di VGG11 preaddestrata

model = FuseModelWrapper(model=torch_model,                                                  #modello di cui si vuole fare il wrapping
                        model_inputs=['data.image'],                                         #sequenza di chiavi nel dizionario dei batch da trasferire alla funzione model.forward
                        post_forward_processing_function=perform_softmax,                    #si sceglie di effettuare una elaborazione di forwarding di tipo SoftMax, usando la funzione apposita
                        model_outputs=['logits.classification', 'output.classification']     #chiavi del dizionario dei batch in cui vado a mettere l'output del modello
                        )

Downloading: "https://download.pytorch.org/models/vgg11-8a719046.pth" to /root/.cache/torch/hub/checkpoints/vgg11-8a719046.pth


  0%|          | 0.00/507M [00:00<?, ?B/s]

### Stampa della conformazione della rete convoluzionale

In [ ]:
torch_model

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (11): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): ReLU(inplace=True)
    (13): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (14): ReLU(inplace=True)
    (15): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
 

## **Fase di creazione della funzione di Loss**
Si crea ora un dizionario di elementi di loss, dove ogni elemento è una classe di tipo FuseLossBase.

Il loss totale è calcolato come somma pesata di tutti gli elementi di tale dizionario.

L'API Fuse estrae la predizione del modello e l'etichetta dal dizionario e poi applica una funzione di calcolo del loss considerandone i pesi definiti dall'utente.

In [ ]:
losses = {'cls_loss': FuseLossDefault(pred_name='model.logits.classification',      #si definisce l'etichetta delle predizioni
                                      target_name='data.label',                     #si sceglie il nome della colonna target
                                      callable=F.cross_entropy,                     #si imposta la funzione di pytorch di calcolo del loss
                                      weight=1.0)                                   #valore da moltiplicare ai pesi finali per computare il loss totale
                                                                                    #serve per dare un peso maggiore ai loss calcolati mano mano durante il processo
}

## **Fase di definizione delle metriche di addestramento**
Si crea un dizionario di elementi, in cui ogni elemento è un metrica definita come oggetto della classe FuseMatricBase.

Le metriche sono calcolate per ogni epoca, sia per la validation che per la fase di addestramento.

La 'best_epoch_source', serve a salvare il miglior modello ottenuto durante la fase di train basandosi sulle metriche che vengono definite.

In [ ]:
metrics = OrderedDict([
    # definisco la soglia da usare per la classificazione, se impostato così si fa ArgMax, con le probabilità
    ('operation_point', MetricApplyThresholds(pred='model.output.classification')),                         #pred: parametro che definisce il nome della chiave nel vettore degli score delle predizioni
                                                                                                            #class_names: nomi delle classi. Questo parametro è richiesto se si fa un problema multiclasse
    #creo l'oggetto Accuracy
    ('accuracy', MetricAccuracy(pred='results:metrics.operation_point.cls_pred',        #chiave delle predizioni da collezionare su cui fare il calcolo
                                target='data.label'))                                   #chiave della classe target su cui calcolare l'accuracy
])

## **Fase di creazione degli oggetti Callbacks**
Definisco i callbacks come oggetti della classe FuseCallbackBase

Un **callback** è un oggetto che fa varie azioni durante i passi del training.

Ad ogni step è possibile infatti fare delle manipolazioni dei dati, del dizionario dei batch batch_dict, o dei risultati di ogni epoca epoch_results.


In [ ]:
callbacks = [
    FuseTensorboardCallback(model_dir=paths['model_dir']),  # la funzione serve per salvare le statistiche di train e validation in dei file di log di tensor
                                                              # detti tensorboard. Devo definire solo il path in cui vengono salvate le cose
]

## **Fase di addestramento della rete**
Si va a costruire un manager di Fuse, e si correda tale manager di ottimizzatori e di scheduler presi dalla libreria Pytorch.

I possibili workflow da seguire sono nella documentazione della classe FuseManagerDefault.

Si nota che il manager usa i parametri di training che abbiamo settato in precedenza.

In [ ]:
# Creo l'ottimizzatore usando Adam, dando in input i parametri del modello, il learning rate e i pesi
optimizer = optim.Adam(model.parameters(), lr=train_params['manager.learning_rate'], weight_decay=train_params['manager.weight_decay'])

# creo lo scheduler sull'ottimizzatore per ridurre il learning rate quando il modello smette di migliorarsi
# lo scheduler vede se l'ottimizzatore migliora, altrimenti abbassa il learing rate e da un miglioramento più fine alla rete
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer)

# tdefinisco il Manager di Fuse, per la gestione dei processi di train e di validation. In questo caso si sta facendo un train by scratch
manager = FuseManagerDefault(output_model_dir=paths['model_dir'],           #path della directory del modello
                             force_reset=paths['force_reset_model_dir'])    #se è True la directory si ripristina in automatico
                                                                            #se è False, cioè di default, la directory va resettata manualmente

#============================================================================================================================================
# I POSSIBILI WORKFLOW DATI DAL MANAGER SONO I SEGUENTI:
#
#     Per l'addestramento:
#         FuseManagerDefault() -> manager.set_objects() -> manager.train()
#     Per riprendere l'addestramento da un checkpoint:
#         FuseManagerDefault() -> manager.load_objects() -> manager.load_checkpoint() -> manager.train()
#     Per l'addestramento partendo da un modello pre-esistente:
#         FuseManagerDefault() -> manager.set_objects() [-> manager.load_objects()] [-> manager.load_checkpoint()] -> manager.train()
#     Per la fase di inferenza:
#         FuseManagerDefault() -> manager.infer()
#         or -
#         FuseManagerDefault() -> manager.load_objects() -> manager.load_checkpoint() -> manager.infer()
#     Per la fase di inferenza dato un modello:
#         FuseManagerDefault() -> manager.set_objects() -> manager.load_checkpoint() -> manager.infer()
#============================================================================================================================================


# Impostiamo il manager per lavorare con gli oggetti che abbiamo creato:
manager.set_objects(net=model,                                                      #modello in input
                    optimizer=optimizer,                                            #ottimizzatore
                    losses=losses,                                                  #definizione delle funzioni di loss  
                    metrics=metrics,                                                #dizionario delle metriche da elaborare per ogni batch
                    best_epoch_source=train_params['manager.best_epoch_source'],    #metriche usate per decidere la best epoch. Può essere anche una lista che contiene le chiavi:
                                                                                    #   'source': nome della metrica di loss- e.g. losses.cls_loss or metrics.auc
                                                                                    #   'optimization': l'ottimizzazione da fare sulla metrica, massimizzare o minimizzare.
                                                                                    #   'on_equal_values': che cosa fare in caso di valori uguali di epoca, prendere il migliore(best) o il peggiore(worse)
                    lr_scheduler=scheduler,                                         #funzione di scheduling
                    callbacks=callbacks,                                            #eventuali callback che voglio fare, nel nostro caso salvare i pesi
                    train_params=train_params['manager.train_params'])              #set di parametri di training che ho definito in un dizionario prima

# FUNZIONE CHE ESEGUE L'ADDESTRAMENTO DELLA RETE PASSANDOGLI IL DATASET DI TRAIN E DI VALIDATION SU CUI CALCOLARE LE METRICHE
manager.train(train_dataloader=train_dataloader, validation_dataloader=validation_dataloader)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
100%|██████████| 7/7 [00:10<00:00,  1.53s/it]
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
100%|██████████| 24/24 [00:23<00:00,  1.00

## **Fase di inferenza**
Una volta addestrata la rete, voglio i suoi migliori parametri per testare la rete su un test set locale, per valutare la capacità di generalizzazione della rete stessa.

### Definizione dei parametri di inferenza

In [ ]:
INFER_COMMON_PARAMS = {}
INFER_COMMON_PARAMS['infer_filename'] = 'validation_set_infer.gz'
INFER_COMMON_PARAMS['checkpoint'] = 'best' 


infer_common_params = INFER_COMMON_PARAMS

### Processo di inferenza

In [ ]:
# prendo dei pezzi dal validation per fare il test locale
torch_test_dataset = {'test': datasets.ImageFolder(os.path.join(data_dir, 'test'), transform)}

test_dataset = FuseDatasetWrapper(name='test', dataset=torch_test_dataset['test'], mapping=('image', 'label'))
# si procede a creare il dataset wrappato
test_dataset.create()

test_dataloader = DataLoader(dataset=test_dataset, collate_fn=test_dataset.collate_fn, batch_size=2, num_workers=2)

# creo un manager Fuse per fare le operazioni di inferenza
manager = FuseManagerDefault()
# definisco le colonne che mi servono per l'output
output_columns = ['model.output.classification', 'data.label']

# FUNZIONE CHE ESEGUE IL PROCESSO DI INFERENZA SUI DATI
manager.infer(data_loader=test_dataloader,                                                                #definizione del dataset di inferenza
                input_model_dir=paths['model_dir'],                                                             #path del modello da dove devo prendere i dati
                checkpoint=infer_common_params['checkpoint'],                                                   #definisco da dove devo prendere i pesi della rete
                output_columns=output_columns,                                                                  #scelgo le colonne che devo restituire in output
                output_file_name=os.path.join(paths["inference_dir"], infer_common_params["infer_filename"]))   #path dove vanno a finire gli output

100%|██████████| 227/227 [00:06<00:00, 32.52it/s]


,descriptor,id,model.output.classification,data.label
0,"(test, 0)","(test, 0)","[0.99999917, 2.1591481e-07, 2.3864287e-11, 2.2...",0
1,"(test, 1)","(test, 1)","[0.9999995, 2.2115915e-07, 9.6838666e-12, 9.01...",0
2,"(test, 2)","(test, 2)","[0.9999999, 3.23253e-08, 1.5215458e-12, 1.4138...",0
3,"(test, 3)","(test, 3)","[0.99999607, 3.1012526e-06, 5.3519227e-11, 4.5...",0
4,"(test, 4)","(test, 4)","[0.9999765, 2.1153464e-05, 1.919223e-10, 1.655...",0
...,...,...,...,...
449,"(test, 449)","(test, 449)","[1.4600917e-06, 0.99999845, 1.2417709e-10, 9.4...",1
450,"(test, 450)","(test, 450)","[1.0245627e-05, 0.99998975, 4.740749e-12, 4.04...",1
451,"(test, 451)","(test, 451)","[0.000512005, 0.999488, 1.5353308e-12, 1.16443...",1
452,"(test, 452)","(test, 452)","[3.101214e-06, 0.9999969, 2.7139688e-11, 1.885...",1


## **Fase di valutazione delle performance**
Uso la classe Evaluator per la valutazione delle performance. Non è necessario che il modello sia di Fuse per usare questa classe.


### Definizione dei parametri di valutazione

In [ ]:
EVAL_COMMON_PARAMS = {}
EVAL_COMMON_PARAMS['infer_filename'] = INFER_COMMON_PARAMS['infer_filename']
eval_common_params = EVAL_COMMON_PARAMS

### Definizione delle metriche di valutazione

In [ ]:
# definisco le classi su cui calcolare le metriche

class_names = ['volGBM', 'volMET']

# Definizione delle metriche come dizionario
metrics = OrderedDict([
    ('operation_point', MetricApplyThresholds(pred='model.output.classification')), # come fatto in precedenza si applica ArgMax
    ('accuracy', MetricAccuracy(pred='results:metrics.operation_point.cls_pred', target='data.label')), # definizione dell'accuracy come classe , come fatto prima
    ('roc', MetricROCCurve(pred='model.output.classification',          #creo una curva ROC per la valutazione e la salvo in un immagine
                           target='data.label', 
                           class_names=class_names, 
                           output_filename=os.path.join(paths['inference_dir'], 'roc_curve.png'))), 
    ('auc', MetricAUCROC(pred='model.output.classification',        # definisco la metrica AUC sulla cuva ROC
                         target='data.label', 
                         class_names=class_names)),
    ('aucpr', MetricAUCPR(pred='model.output.classification',
                                  target='data.label',
                                  class_names=class_names)),
    ('brier-skill', MetricBSS(pred='model.output.classification',
                                  target='data.label'))
])

### Processo di valutazione e visualizzazione dei risultati

In [ ]:
# creo la classe Evaluator
evaluator = EvaluatorDefault()

# FUNZIONE CHE ESEGUE IL PROCESSO DI EVALUATION SUI DATI
results = evaluator.eval(ids=None,
                data=os.path.join(paths["inference_dir"], eval_common_params["infer_filename"]),
                metrics=metrics,
                output_dir=paths['eval_dir'])

Results:

Metric operation_point:
------------------------------------------------
cls_pred:

Metric accuracy:
------------------------------------------------
0.9537444933920705

Metric roc:
------------------------------------------------
volGBM.fpr:
[0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.00552486 0.00552486
 0.00552486 0.00552486 0.01104972 0.01104972 0.01657459 0.01657459
 0.02209945 0.02209945 0.02762431 0.02762431 0.03314917 0.03314917
 0.0441989  0.0441989  0.04972376 0.04972376 0.06077348 0.06077348
 0.06629834 0.06629834 0.0718232  0.0718232  0.08287293 0.08287293
 0.09392265 0.09392265 0.09944751 0.09944751 0.11049724 0.11049724
 0.12707182 0.12707182 0.14364641 0.14364641 0.15469613 0.15469613
 0.68508287 0.6961326  1.        ]
volGBM.

#### Brier-Skill Score

L'**indice di Brier** indica la percentuale di incertezza del classificatore. E' un valore tra 0 e 1, e più vicino a zero è più è bassa l'incertezza del classificatore.
La metrica che calcola Fuse è invece il complemento a 1 della percentuale dell'indice di brier, e quindi più è alto il **punteggio Brier-Skill**, minore sarà l'incertezza del classificatore.

Praticamente sto calcolando lo scarto quadratico medio delle predizioni. 